## Web Scraping of Estimize.com data using Selenium with Python

In [2]:
# Importing all the required libraries
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from csv import writer
import pandas as pd
import time
import csv 


## List of all the tickers

In [9]:
ticker_list = pd.read_csv('ticker_list.csv') # reading URLs of all tickers from a csv file
ticker_list.head()

ticker_url = ticker_list['URL'] # URL of the ticker
ticker_name = ticker_list['ticker'] # Name of the ticker

In [ ]:
ticker_url.head()

In [ ]:
ticker_name.head()

## First Module for company basic information such as Ticker, Company Name, Sectors, Industries, Number of Followers, Number of Analysts

In [ ]:
driver = webdriver.Chrome(executable_path="D:\GSU\Fall 2021\Data Management\chromedriver.exe")  #Chormedriver path

def get_basic_info(url,i):  # Defining a function

    driver = webdriver.Chrome(executable_path="D:\GSU\Fall 2021\Data Management\chromedriver.exe")

    driver.get(url)
    basic_info_list = [] #Initiating an empty list

    ticker = ticker_name[i]
    basic_info_list.append(ticker)
    try:
        name= driver.find_element_by_class_name('release-header-information-description').text
        basic_info_list.append(name)
    except NoSuchElementException:
        basic_info_list.append('N/A')

    try:    
        sectors_industries_list = driver.find_element_by_class_name('release-header-information-breadcrumb').text.split('›')
        sector = sectors_industries_list[0]
        basic_info_list.append(sector)
    except NoSuchElementException:
        basic_info_list.append('N/A')

    try:   
        sectors_industries_list = driver.find_element_by_class_name('release-header-information-breadcrumb').text.split('›')
        industry = sectors_industries_list[1]
        basic_info_list.append(industry)
    except NoSuchElementException:
        basic_info_list.append('N/A')  

    try:      
        header = driver.find_element_by_class_name('content-header-show')
        nav = header.find_element_by_id('summary-stats')
        followers = nav.find_element_by_xpath('//*[@id="summary-stats"]/div/div/div[1]/div[2]').text
        basic_info_list.append(followers)
    except NoSuchElementException:
        basic_info_list.append('N/A')

    try:
        header = driver.find_element_by_class_name('content-header-show')
        nav = header.find_element_by_id('summary-stats')
        analysts = nav.find_element_by_xpath('//*[@id="summary-stats"]/div/div/div[2]/a').text
        basic_info_list.append(analysts)
    except NoSuchElementException:
        basic_info_list.append('N/A')


    driver.close()

    return basic_info_list

for i in range(len(ticker_url)): #looping over all tickers
        
        # driver = webdriver.Chrome(executable_path="D:\GSU\Fall 2021\Data Management\chromedriver.exe")
    url = ticker_url[i]
    basic_info_list = get_basic_info(url,i) #calling the function
    
    with open('basic_info.csv', 'a', newline="", encoding='utf8') as f_object:  # saving the scraped data into a csv file
        writer_object = writer(f_object)
        writer_object.writerow(basic_info_list)

        f_object.close() 

In [13]:
# Getting data quarter-wise for the FY 2020
q1 = "/fq1-2020?metric_name=eps&chart=table"
q2 = "/fq2-2020?metric_name=eps&chart=table"
q3 = "/fq3-2020?metric_name=eps&chart=table"
q4 = "/fq4-2020?metric_name=eps&chart=table"

ticker_url_q = []

for i in range(len(ticker_url)):
    ticker_url_q.append(ticker_url[i]+q1)
    ticker_url_q.append(ticker_url[i]+q2)
    ticker_url_q.append(ticker_url[i]+q3)
    ticker_url_q.append(ticker_url[i]+q4)

In [ ]:
ticker_url_q.head()

## Second Module for EPS Analysts and EPS info

In [ ]:
def get_eps_info(url,i): # defining a function for Reported Earnings, Estimize Consensus, Estimize Mean, Wall Street Consensus
    driver = webdriver.Chrome(executable_path="D:\GSU\Fall 2021\Data Management\chromedriver.exe")
    driver.get(url)
    eps_info_list = [] #initiating an empty list
    quarter = url.split('/')[4].split('?')[0]
    ticker = ticker_name[i] 
    eps_info_list.append(ticker)
    eps_info_list.append(quarter)
    try:
        reported_earnings= driver.find_element_by_xpath('//*[@id="estimates"]/table/tbody[1]/tr[1]/td[5]').text
        eps_info_list.append(reported_earnings)
    except NoSuchElementException:
        eps_info_list.append('N/A')
    
    try:
        estimize_concensus= driver.find_element_by_xpath('//*[@id="estimates"]/table/tbody[1]/tr[2]/td[5]').text
        eps_info_list.append(estimize_concensus)
    except NoSuchElementException:
        eps_info_list.append('N/A')
    
    try:
        estimize_mean= driver.find_element_by_xpath('//*[@id="estimates"]/table/tbody[1]/tr[3]/td[5]').text
        eps_info_list.append(estimize_mean)
    except NoSuchElementException:
        eps_info_list.append('N/A')
    
    try:
        ws_concensus= driver.find_element_by_xpath('//*[@id="estimates"]/table/tbody[1]/tr[4]/td[5]').text
        eps_info_list.append(ws_concensus)
    except NoSuchElementException:
        eps_info_list.append('N/A')
        
    driver.close()
    return eps_info_list


#get_analyst_eps_info_list('https://www.estimize.com/fsct/fq1-2020?metric_name=eps&chart=historical',1)
i = 0  #initializing i to 0
for j in range(len(ticker_name)): # looping over all tickers
    lo_lim = i
    up_lim = i+4
    for k in range(lo_lim,up_lim):
        url = ticker_url_q[k]
        eps_info_list = get_eps_info(url,j) #calling the function 
    
        with open('eps_info.csv', 'a', newline="", encoding='utf8') as f_object: # saving the scraped data into a csv file
            writer_object = writer(f_object)
            writer_object.writerow(eps_info_list)

            f_object.close()
            if ((k+1) % 4 == 0):
                i = up_lim
                break;

In [ ]:
def get_analyst_eps_info_table(url,i): #defining the function for EPS estimations of all available analysts
    driver = webdriver.Chrome(executable_path="D:\GSU\Fall 2021\Data Management\chromedriver.exe")
    driver.get(url)
    
    analyst_eps_info_table = [] # creating an empty list
    ticker = ticker_name[i] # creating an empty list

    try:
        eps_analyst_table = driver.find_element_by_xpath('//*[@id="estimates"]/table').text
    except NoSuchElementException:
        pass
    driver.close()
    eps_analyst_table = eps_analyst_table.split('Show Estimate')[1:]
    return eps_analyst_table

i = 0 
analyst_eps_table = pd.DataFrame(columns=['ticker','quarter','username','name','rank','points','estimate','confidence','last_revised'])
for j in range(len(ticker_name)): #looping over all tickers
    lo_lim = i
    up_lim = i+4
    for k in range(lo_lim,up_lim):
        url = ticker_url_q[k]
        analyst_eps_info_list = get_analyst_eps_info_table(url,j)
        if not analyst_eps_info_list:
            analyst_eps_table.loc[len(analyst_eps_table),:] = [ticker_name[j],url.split('/')[4].split('?')[0],'N/A','N/A','N/A','N/A','N/A','N/A','N/A']
            if ((k+1) % 4 == 0):
                i = up_lim
                break;
        else:
            for m in range(len(analyst_eps_info_list)):
                final_list = []
                quarter = url.split('/')[4].split('?')[0]
                ticker = ticker_name[j]
                final_list.append(ticker)
                final_list.append(quarter)

                try:
                    eps_analyst_username = analyst_eps_info_list[m].split('\n')[3]
                    final_list.append(eps_analyst_username)
                except:
                    final_list.append('N/A')

                try:
                    eps_analyst_name = analyst_eps_info_list[m].split('\n')[2]
                    final_list.append(eps_analyst_name)
                except:
                    final_list.append('N/A')

                try:
                    eps_analyst_rank = int(analyst_eps_info_list[m].split('\n')[4].split(' ')[0])
                    final_list.append(eps_analyst_rank)
                except:
                    final_list.append('N/A')

                try:
                    eps_analyst_points = int(analyst_eps_info_list[m].split('\n')[4].split(' ')[1])
                    final_list.append(eps_analyst_points)
                except:
                    final_list.append('N/A')

                try:
                    eps_analyst_estimate = float(analyst_eps_info_list[m].split('\n')[4].split(' ')[2])
                    final_list.append(eps_analyst_estimate)
                except:
                    final_list.append('N/A')

                try:
                    eps_analyst_confidence = analyst_eps_info_list[m].split('\n')[5].split(' ')[0]
                    final_list.append(eps_analyst_confidence)
                except:
                    final_list.append('N/A')

                try:
                    eps_analyst_revised = analyst_eps_info_list[m].split('\n')[5].split(' ')[1]
                    final_list.append(eps_analyst_revised)
                except:
                    final_list.append('N/A')

                analyst_eps_table.loc[len(analyst_eps_table),:] = final_list
                    
                if ((k+1) % 4 == 0):
                    i = up_lim
                    break; 

#Exporting dataframe to CSV
analyst_eps_table.to_csv('analyst_eps_table.csv')

In [91]:
#analyst_eps_table = pd.read_csv('analyst_eps_table.csv')
analyst_eps_table['username'] = analyst_eps_table['username'].astype(str).dropna()
analyst_username = analyst_eps_table['username'].unique().tolist()
analyst_username = list(map(lambda x: x.lower(), analyst_username))
link_prefix = "https://www.estimize.com/users/"

analyst_url = []

for i in range(len(analyst_username)):
    analyst_url.append(link_prefix+analyst_username[i])

In [92]:
analyst_username = analyst_username[1:]
analyst_url = analyst_url[1:]

In [94]:
analyst_url.remove('https://www.estimize.com/users/nan')

In [97]:
analyst_username.remove('nan')

In [12]:
analyst_username = pd.read_csv('D:\GSU\Fall 2021\Data Management\AnalystsUsername.csv') #Getting analyst username

analyst_url = pd.read_csv('D:\GSU\Fall 2021\Data Management\AnalystsURL.csv') #Getting analyst URL

## Third Module for getting details of all analysts

In [ ]:

def get_analyst_info(url,i):
    driver = webdriver.Chrome(executable_path='D:\GSU\Fall 2021\Data Management\chromedriver')
    #driver.set_page_load_timeout(30)
    driver.get(url)
    #unhash the following two lines if the page takes long time to load
    #delay=30
    #element_loc = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID ,'profile-covered-stocks')))
    analyst_user = analyst_username[i]
    analyst_info_list = []
    analyst_info_list.append(analyst_user)
    #/html/body/div[4]/div[2]/div/div[2]/div[2]/div[3]/div[4]/table

    try: 
        analyst_name = driver.find_element_by_class_name('profile-display-name').text
        analyst_info_list.append(analyst_name)
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_role = driver.find_element_by_class_name('profile-bio-categorizations').text
        analyst_info_list.append(analyst_role)
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_join_date = driver.find_element_by_class_name('profile-activity-stats').text.split('since ')[1]
        if '-' in analyst_join_date:
            analyst_info_list.append(analyst_join_date.split('-')[0])
        else:
            analyst_info_list.append(analyst_join_date)
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_confidence = driver.find_element_by_xpath('//*[@id="confidence-wrap"]/div/div[2]').text
        if analyst_confidence == '-' or analyst_confidence == 'N/A':
            analyst_info_list.append('N/A')
        else:
            analyst_info_list.append(float(analyst_confidence))
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_error = driver.find_element_by_xpath('//*[@id="profile-tab-wrap"]/div[1]/div[1]/div[1]').text
        if analyst_error == '-' or analyst_error == 'N/A':
            analyst_info_list.append('N/A')
        else:
            analyst_info_list.append(analyst_error)
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_accuracy = driver.find_element_by_xpath('//*[@id="profile-tab-wrap"]/div[1]/div[2]/div[1]').text
        analyst_info_list.append(analyst_accuracy)
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_points = driver.find_element_by_xpath('//*[@id="profile-tab-wrap"]/div[2]/div[1]/div[1]').text.replace(",", "")
        if analyst_points == '-':
            analyst_info_list.append('N/A')
        else:
            analyst_info_list.append(float(analyst_points))
    except NoSuchElementException:
        analyst_info_list.append('N/A')


    try:
        analyst_points_estimate = driver.find_element_by_xpath('//*[@id="profile-tab-wrap"]/div[2]/div[2]/div[1]').text.replace(",", "")
        if analyst_points_estimate == '-':
            analyst_info_list.append('N/A')
        else:
            analyst_info_list.append(float(analyst_points_estimate))
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_stocks = driver.find_element_by_xpath('//*[@id="profile-tab-wrap"]/div[3]/div[1]/div[1]').text.replace(",", "")
        if analyst_stocks == '-':
            analyst_info_list.append('N/A')
        else:
            analyst_info_list.append(float(analyst_stocks))
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    try:
        analyst_pen_est = driver.find_element_by_xpath('//*[@id="profile-tab-wrap"]/div[3]/div[2]/div[1]').text.replace(",", "")
        if analyst_pen_est == '-':
            analyst_info_list.append('N/A')
        else:
            analyst_info_list.append(float(analyst_pen_est))
    except NoSuchElementException:
        analyst_info_list.append('N/A')

    driver.close()
    return analyst_info_list

analyst_info_table = pd.DataFrame(columns=['Username','Name','Role','Join Date','Analyst Confidence Score','Error Rate','Accuracy Percentile','Points','Points/Estimate','Stocks','Pending'])

for i in range(len(analyst_url)):
    url=analyst_url[i]
    analyst_information = get_analyst_info(url,i)
    analyst_info_table.loc[len(analyst_info_table),:] = analyst_information
    
#Exporting dataframe to csv
#analyst_info_table.to_csv('analyst_info_table.csv')

## Fourth Modeule for all covered stock estimates by the analyst


In [ ]:
from selenium import webdriver
browser1 = webdriver.Chrome(executable_path=r'D:\GSU\Fall 2021\Data Management\chromedriver')
analyst_list = pd.read_csv('unique_usernames.csv').username.tolist()

analysts = []
tickers = []
reports = []
quarters = []
points = []
pt_sts = []
err_rates = []
accuracies = []

print(len(analyst_list))
num = 0

for analyst in analyst_list:
    num = num+1
    print(num)
    url = 'https://www.estimize.com/users/'+analyst
    browser1.get(url)
    
    browser1.execute_script("window.scrollTo(0, window.scrollY + 1200)") #scrolling to a particular height
    time.sleep(2)
    
    for a in range(0,1): #if there are no element in the stocks covered table.
        a = a+1
        try:
            #print("checking the show more button")
            if(browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[2]/a')):
                #print("in here 1") //*[@id="profile-covered-stocks"]/div[1]/div[2]/div[1]/div[3]
                a = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[2]/a')
                #print(a)
                a.click()

        except NoSuchElementException: 
            break
        
    
    for j in range(1,31):
        
        for a in range(0,1): #if there are no element in the stocks covered table.
                a = a+1
                try:
                    if(browser1.find_element(By.XPATH,'//*[@id="profile-scored-estimates"]/div[1]/div[2]/a['+str(j)+']/div[1]')):
                        #print("in here 2")
                        #print(j)
                        ticker = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[1]/div[2]/div['+str(j)+']/div[1]/a').text
                        #print(ticker)
                        quarter = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[1]/div[2]/div['+str(j)+']/div[3]').text
                        point = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[1]/div[2]/div['+str(j)+']/div[4]').text
                        pt_st = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[1]/div[2]/div['+str(j)+']/div[5]').text
                        err_rate = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[1]/div[2]/div['+str(j)+']/div[6]').text
                        accuray = browser1.find_element(By.XPATH,'//*[@id="profile-covered-stocks"]/div[1]/div[2]/div['+str(j)+']/div[7]').text
                        
                        analysts.append(analyst)
                        tickers.append(ticker)
                        quarters.append(quarter)
                        points.append(point)
                        pt_sts.append(pt_st)
                        err_rates.append(err_rate)
                        accuracies.append(accuray)
                        
                except NoSuchElementException:
                    break
                
    

In [96]:
df = pd.DataFrame()
df['analyst']=analysts
df['tickers']=tickers
df['quarters']=quarters
df['points']=points
df['pt_sts']=pt_sts
df['err_rates']=err_rates
df['accuracies']=accuracies

df.to_csv('stocks_covered.csv')



## Fifth modeule for all pending stock estimates by the analyst


In [ ]:
from selenium import webdriver
# driver = webdriver.Chrome(executable_path=r'D:\GSU\Fall 2021\Data Management\chromedriver')
browser1 = webdriver.Chrome(executable_path=r'D:\GSU\Fall 2021\Data Management\chromedriver')
analyst_list = ['canaccord_146','billb1210']
# analyst_list = pd.read_excel('df_analyst_new.xlsx').Username.tolist()
#analyst_list = pd.read_excel('df_analyst_new_1.xlsx').Username.tolist()

#analyst_list = analyst_list[0:3]

analysts=[]
tickers = []

quarters = []
epss = []
revenues = []
# reportss = []
# publisheds = []

print(len(analyst_list))
num = 0
                

for analyst in analyst_list:
    url = 'https://www.estimize.com/users/'+analyst
    browser1.get(url)
    
    browser1.execute_script("window.scrollTo(0, window.scrollY + 1200)") #scrolling to a particular height
    time.sleep(3)
    
    for a in range(0,1): #if there are no element in the stocks covered table.
        a = a+1
        try:
            print("checking the show more button")
            if(browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[2]/a[2]')):
                print("in here 1")
                a = browser1.find_element(By.XPATH,'//*[@id="profile-pending-stocks"]/div[2]/a[2]')
                print(a)
                a.click()
        except NoSuchElementException:
            break
        
    
    for j in range(1,31):
        
        for a in range(0,1): #if there are no element in the stocks covered table.
                a = a+1
                try:
                    # //*[@id="profile-pending-estimates"]/div[1]/div[2]/a[1]
                    if(browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[1]/strong')):
                        #print("in here 2")
                        #print(j)
                        ticker = browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[1]/strong').text
                        #print(ticker)
                        quarter = browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[2]').text
                        #print(quarter)
                        # reports = browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[3]').text
                        # published = browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[4]').text

                        eps = browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[5]').text
                        #print(eps_point)
                        revenue = browser1.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]/a['+str(j)+']/div[6]').text
                        #print(revenue_point)
                        
                        #print(total_point)

                        
                        analysts.append(analyst)
                        tickers.append(ticker)
                        #reports.append(report)
                        quarters.append(quarter)
                        epss.append(eps)
                        revenues.append(revenue)
                        # reportss.append(reports)
                        # publisheds.append(published)
                        #err_rates.append(err_rate)
                        #accuracies.append(accuray)
                        
                except NoSuchElementException:
                    break
    

In [ ]:
df2 = pd.DataFrame()
df2['analyst']=analysts
df2['tickers']=tickers
df2['quarters']=quarters
# df2['reports'] = reports
# df2['publisheds'] = publisheds
df2['eps']=epss
df2['revenue']=revenues


# df2
df2.to_csv('penidng_stocks.csv')

## Sixth module for all scored stock estimates by the analyst


In [101]:

time.sleep(0.5)
scored_list = []
# loop through each analyst page
browser = webdriver.Chrome(executable_path=r'D:\GSU\Fall 2021\Data Management\chromedriver')
analyst_list_2 = pd.read_csv('unique_usernames.csv').username.tolist()
for i in range(0,len(analyst_list_2)):

    url = 'https://www.estimize.com/users/' + analyst_list_2[i]
    browser.get(url)
    # count total scored estimate    
    try:
        count = int(browser.find_element(By.XPATH,'//*[@id="equities"]/div[1]/div[4]/p/strong/span[2]').text)
    except:
        count = 0
            
    if count == 0:
        insert_row = []
        insert_row.extend([analyst_list_2[i],'', '', '', '', '', '' ])
        scored_list.append(insert_row)
    else:
        # click show 20 more
        try:
            click_list = browser.find_elements(By.XPATH, '//*[@id="profile-scored-estimates"]/div[2]/a[2]')
            click_list[0].click()
        except:
            pass
    
        try:
            click_list = browser.find_elements(By.XPATH, '//*[@id="profile-scored-estimates"]/div[2]/a[1]')
            click_list[0].click()
        except:
            pass
        
        try:
            click_list = browser.find_elements(By.XPATH, '//*[@id="profile-scored-estimates"]/div[2]/a')
            click_list[0].click()
        except:
            pass
        # loop through scored est. table
        for j in range(1, min(20, count)+1):  
            insert_row = []
              
            ticker = browser.find_element(By.XPATH,' //*[@id="profile-scored-estimates"]/div[1]/div[2]/a['+
                                                        str(j) + ']/div[1]/strong').text
            Quarter = browser.find_element(By.XPATH,'//*[@id="profile-scored-estimates"]/div[1]/div[2]/a[' + 
                                                        str(j)+ ']/div[2]').text    
            Rank = browser.find_element(By.XPATH,'//*[@id="profile-scored-estimates"]/div[1]/div[2]/a[' +
                                                        str(j) + ']/div[4]').text 
            EPS_Points = browser.find_element(By.XPATH,'//*[@id="profile-scored-estimates"]/div[1]/div[2]/a[' +
                                                       str(j) + ']/div[5]').text
            Revenue_Points = browser.find_element(By.XPATH,'//*[@id="profile-scored-estimates"]/div[1]/div[2]/a['+
                                                       str(j) + ']/div[6]').text
            Total_Points = browser.find_element(By.XPATH,'//*[@id="profile-scored-estimates"]/div[1]/div[2]/a['+
                                                       str(j) + ']/div[7]').text
                
            insert_row.extend([analyst_list_2[i],ticker, Quarter, Rank, EPS_Points,
                              Revenue_Points,Total_Points])
            scored_list.append(insert_row)
    
pd.DataFrame(scored_list)

,0,1,2,3,4,5,6
0,quanttrader007,FLR,Q2 2020,11 / 22,-3,11,8
1,quanttrader007,MLCO,Q2 2020,24 / 25,-4,-25,-29
2,quanttrader007,WOLF,Q4 2020,32 / 37,-4,-6,-10
3,quanttrader007,MDP,Q4 2020,9 / 10,-6,-7,-13
4,quanttrader007,ICON,Q2 2020,3 / 3,-6,-4,-10
...,...,...,...,...,...,...,...
3052,analyst_1729238,TAP,Q2 2021,15 / 15,-9,-25,-34
3053,analyst_1729238,AAPL,Q3 2021,14 / 348,19,24,43
3054,analyst_1729238,DHR,Q2 2021,10 / 11,-6,-7,-13
3055,analyst_1729238,FDX,Q4 2021,25 / 27,-25,-18,-43


In [109]:
scored.to_csv('last_table.csv')